In [25]:

def  fnlpprint():
    import nlp.NLP
    
    
#     wav--------------------------------讀檔（wav）
    
    audio_path = r"nlp/test_audio/_67485707.wav"


    # m4a---------------------------讀檔（m4a）
    

#     path = r"nlp/test_audio/_67559426.m4a"
#     path = r"nlp/test_audio/_67633155.m4a"    
#     path = r"nlp/test_audio/_67887224.m4a"
#     path = r"nlp/test_audio/_67903525.m4a"
    # path = r"nlp/test_audio/_67903526.m4a"
#     path = r"nlp/test_audio/_67903527.m4a" 
#     path = r"nlp/test_audio/_67903528.m4a"
#     path = r"nlp/test_audio/_67903530.m4a"   
#     path = r"nlp/test_audio/_67903531.m4a"
#     path = r"nlp/test_audio/_67903532.m4a" 
#     path = r"nlp/test_audio/_67903533.m4a"
#     path = r"nlp/test_audio/_67936273.m4a"
#     NLP.m4a_to_wav(path)  # m4a轉成wav


#     audio_path = r'nlp/test_audio/speak.wav'





    sentence = nlp.NLP.speech_to_text(audio_path)  #音檔轉文字
    
    nlpprint=[]
    choose_product = int(0)
    if sentence==[]:                        #無聲音
        nlpprint.append("無")
        choose_product=0
    else:
        seg_list = nlp.NLP.cut_word(sentence)  #斷詞後無內容
        if seg_list==[]:
            nlpprint.append("無")
            choose_product=0
        else:
            product_name=nlp.NLP.get_category(seg_list)                # get_category  提取商品類名
            # print("get_category後",product_name)  # 找出商品名稱

            if((product_name)==[]): 
                product_name=nlp.NLP.get_category2(seg_list)               # get_category2 提取商品類名
                # print("get_category2後",product_name)

                if(product_name==[]): 
                    nlpprint.append("無")                                # 提取後無身品
                    choose_product=0
            else:        
                if(len(product_name)==1):                     # 提取後1筆商品        
                    choose_product=1
                    nlpprint=(product_name[0])  
                else:                                                   # 提取後n筆商品
                    # nlpprint.append("which product is your choose")
                    choose_product=2
                    for i in product_name:
                        nlpprint.append(i)
                        # print(i)    #--待寫---
                        
                    e='1'   #------------假設user選第二個----待寫swift input flask-----
                    e=int(e)
                    product_name[0]=product_name[e]
                    print(product_name[0])
        
    
    
    return nlpprint,choose_product



In [26]:
nlpprint,choose_product=(fnlpprint())
print(choose_product,nlpprint)

0 ['無']


In [27]:
def ocrprint(intn):               
    import ocr.OCR
    import os
    from google.cloud import vision
    import db.pymysql_func
    # key
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "pubkey_group-five-project-4fbbbf03858d.json"

    YOUR_PIC = './uploads/{}'.format(intn)    
    texts = ocr.OCR.pic_to_text(YOUR_PIC)  # 圖片轉文字
    ocr_list=ocr.OCR.get_element(texts)

    disease= {"新陳代謝症候群":[], # disease= 疾病：預存成分
             "糖尿病":[],
             "高血脂":[],
             "高血壓":[],
             "腎臟病":[],
             "癌症":[],
             "word":[texts]}
    

    for key in  disease:         #將疾病對應之高風險添加物加入disease
            
        for e in ocr_list:
            outdisease=db.pymysql_func.find_disease(e)
            for i in outdisease:     
                
                if  i == key:
                    disease[key].append(e)
            # print(disease[key])
            
            
        if(bool(disease[key])==0):#無高風險添加物的疾病value為'無'
            disease[key]=['無']
                    
                    
                    
    disease['syndrome']=disease.pop('新陳代謝症候群')   #中文key改英文
    disease['diabeHyperlipidemiates']=disease.pop('糖尿病')
    disease['Hyperlipidemia']=disease.pop('高血脂')
    disease['hypertension']=disease.pop('高血壓')
    disease['kidney_disease']=disease.pop('腎臟病')
    disease['cancer']=disease.pop('癌症')
   
    
    
    
    disease_additives={'disease_additives':[{'syndrome':disease['syndrome']},   #改變格式
                                            {'diabeHyperlipidemiates':disease['diabeHyperlipidemiates']},
                                            {'Hyperlipidemia':disease['Hyperlipidemia']},
                                            {'hypertension':disease['hypertension']},
                                            {'kidney_disease':disease['kidney_disease']},
                                            {'cancer':disease['cancer']},
                                            {'word':disease['word']}]}
    

    # print('a')

    with open('./templates/tem_output.txt',mode='r+', encoding='utf-8') as f:
        #開啟文件   
        
        print('aaa',disease_additives)  
        message =str(disease_additives)      #改寫&存取txt
            
            
        # 先读取文件, 将文件指针指向开始，并使用truncate()清除所有内容
        data = f.read()
        # print(data)
        f.seek(0)
        f.truncate()
        f.write(message)
        # print(f.read())
        print(message)
        f.close()
        # print('c')

    # return disease_additives

In [ ]:
import json 
from __future__ import unicode_literals
import os      # For File Manipulations like get paths, rename
from flask import Flask, flash, request, redirect, render_template, abort, Response, g, session, redirect, url_for
from werkzeug.utils import secure_filename

import tensorflow as tf
import keras

app = Flask(__name__, static_url_path='/uploads',static_folder='./uploads') #預設圖片儲存資料夾

app.secret_key = "secret key" # for encrypting the session
#It will allow below 16MB contents only, you can change it
# app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024 
path = os.getcwd()
# file Upload
UPLOAD_FOLDER = os.path.join(path, 'uploads')

if not os.path.isdir(UPLOAD_FOLDER):
    os.mkdir(UPLOAD_FOLDER)

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
ALLOWED_EXTENSIONS = set(['txt','png','docx','jpg','wav'])  #允許上傳格式

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

#頁面內容
@app.route('/')
def upload_form():
    return render_template('upload.html')


# 打開存好的疾病與高風險添加物清單,str轉dict轉json後輸出
@app.route('/ocr/res', methods=['POST', 'GET'])
def show_res():
    with open('./templates/tem_output.txt',mode='r+', encoding='utf-8') as f:                           #astar code
        
        data = f.read()
        print(data)
        f.close()  
    return json.dumps((data),ensure_ascii=False),200,{'content-type':'application/json; charset=utf-8'}

#圖片輸出
@app.route('/ocr/img', methods=['POST', 'GET'])
def show_img():
    import glob
    files_grabbed = [glob.glob(r"./uploads/*") for _ in ['*.jpg', '*.png']]
    latest_file = max(files_grabbed[0], key=os.path.getctime)  
    head, tail = os.path.split(latest_file)                                         
    return render_template('show_image.html', img_stream="http://34.72.27.77:5000/uploads/{}".format(tail))


@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html')


        
    

#上傳圖片、音檔
@app.route('/',methods=['POST', 'GET'])
def upload_file():
    if request.method == 'POST':
        file = request.files['file']
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            outocr=ocrprint(file.filename)
            return redirect('/ocr/res')   #跳轉
        else:
            return redirect('page_not_found')

    


#最低價五項商品
@app.route('/crawler/low/<instr>',methods=['GET'])
def crawler_low(instr):
    import crawler.price_low_high
    search=crawler.price_low_high.low_to_high(instr) 
    return json.dumps(search,ensure_ascii=False),200,{'content-type':'application/json; charset=utf-8'}

#最高價五項商品
@app.route('/crawler/high/<instr>',methods=['GET'])
def crawler_high(instr):
    import crawler.price_high_low
    search=crawler.price_high_low.high_to_low(instr) 

    return json.dumps(search,ensure_ascii=False),200,{'content-type':'application/json; charset=utf-8'}

#推薦五項商品
@app.route('/crawler/recommend/<instr>',methods=['GET'])

def crawler_recommend(instr):
    import crawler.crawler_product
    search=crawler.crawler_product.goods_info(instr) 

    return json.dumps(search,ensure_ascii=False),200,{'content-type':'application/json; charset=utf-8'}



# @app.route("/nlp_out",methods=['GET'])
# def testnlp_out():
#     nlpprint,choose_product=fnlpprint()


# @app.route("/10",methods=['GET'])
# def fnlp():
#     username = request.args.get("username")
#     userage = request.args.get("userage")    


#     # return "Hello {},you are {} years old.".format(username,userage)
#     return str(fnlpprint())


# @app.route("/hello_post",methods=["POST"]) 
# def hello_post():
#     outStr="""
#     <form action="/hello_post" method="POST">
#         <label>What's your name?</label>
#         <input name="username">
#         <button>SUBMIT</botton>
#     </form>
#     """
#     method=request.method #get or post
#     if method=="POST":
#         username=request.form.get("username")
#         outStr += """
#         <h1>Hello {}.<h1> 
#         """.format(username)


#     return outStr   




if __name__ == "__main__":
    app.run(host='0.0.0.0',port=5000,debug=False,threaded=True)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://172.18.0.3:5000/ (Press CTRL+C to quit)


In [ ]:
import db.pymysql_func

# a=pymysql_func.find_carrefour_goods('巧克力')
# b=pymysql_func.find_online_category('汽水')
c=db.pymysql_func.find_disease('砂糖')
# d=pymysql_func.distinct_xy('巧克力')
print('--------')
# print(a)
# print(b)
print(c)
# print(d)
